# k-fold
- 교차 검증을 통해 모델의 과적합 정도를 판단할수 있습니다.
- https://scikit-learn.org/stable/modules/cross_validation.html

- 장점
    - 특정 데이터셋이대한 과적합 방지
    - 더욱 일반화된 모델 생성 가능
    - 데이터셋 규모가 적을때 과소적합 방지
- 단점
    - 모델 훈련 및 평가 소요시간 증가(반복학습 횟수 증가)

## 1. 데이터 로드

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("train.csv")
df.tail(1)

## 2. 데이터 전처리

In [ ]:
df = pd.read_csv("train.csv")

# 필요한 컬럼만 필터링
columns = ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
df = df[columns]

# 결측 데이터 제거
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# sex, embarked : 더미변수화
dummy_sex = pd.get_dummies(df["Sex"])
dummy_embarked = pd.get_dummies(df["Embarked"])
df = pd.concat([df, dummy_sex, dummy_embarked], axis=1).drop(columns=["Sex", "Embarked"])

df.tail(1)

## 3. 데이터셋 나누기

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_x = df.drop(columns=["Survived"])
df_y = df["Survived"]

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(df_x, df_y, test_size=0.2, random_state=1)
len(train_x), len(test_x), len(train_y), len(test_y)

## 4. 모델링

### 4.1 decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
import numpy as np

In [ ]:
dt_model = DecisionTreeClassifier(random_state=0).fit(train_x, train_y)
score = dt_model.score(test_x, test_y) * 100
np.round(score, 2)

### 4.2 random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_model = RandomForestClassifier(random_state=0).fit(train_x, train_y)
score = rf_model.score(test_x, test_y) * 100
np.round(score, 2)

## 5. k-fold cross validation score
- https://scikit-learn.org/stable/modules/cross_validation.html
- 분산이 크면 데이터에 따라서 모델의 정확도 차이가 크므로 과적합의 위험이 크다.
- 모델링 결과에서 Decision Tree 알고리즘이 정확도가 더 높게 나왔지만 crooss validation score로 확인해보면 random forest가 더 높다.
- Decision Tree 모델의 결과가 과적합될 가능성이 크다.

In [ ]:
from sklearn.model_selection import cross_val_score

### 5.1 decision tree

In [ ]:
scores = cross_val_score(dt_model, test_x, test_y, cv=10)
scores.mean(), scores.var()

### 5.2 random forest

In [ ]:
scores = cross_val_score(rf_model, test_x, test_y, cv=10)
scores.mean(), scores.var()